In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
import zillow
import re

pd.options.display.max_rows = 700

### Instantiate the Zillow API
After installing Zillow API in your terminal you should be able to load the zillow library and the valuation API.  Notice that you must input your zillow api key before moving ahead.  The Zillow api takes your personal key as it's first parameter input and will not run without one.  According to the zillow ToS you must use your own personal key and cannot share with other members of your team/company.

In [4]:
from zillow import ValuationApi

#insert your API key from zillow here
zid = 'X1-ZWz18cqbc8wgsr_7j3qs'

api = zillow.ValuationApi()

Read in your csv Data.  This will be important later for utilization in the code.  If you change the name of your dataframe here you will have to change it in all of the code below

In [5]:
rochester = pd.read_csv('./us/ny/city_of_rochester.csv')

### Cleaning Data
Below are the few steps taken to clean the data.  The first is a simple drop/drop na, removing non-essential columns and removing rows with NaN information (notably, 'STREET' or street names and 'NUMBERS' or house numbers)

In [6]:
rochester.drop(['DISTRICT','REGION','ID','HASH','UNIT'], axis = 1, inplace = True)
rochester.dropna(inplace = True)

This loop runs through our data to replace any 'NUMBER' values seperated in the '120-124' format.  This allows us to search each individual property individually in the API. Be patient, this could take some time to complete.  Note the progress bar, this is a personal preference to track the loops progress.  If this does not run for you, simply run 'pip install progressbar2' in your terminal

In [8]:
for each in rochester['NUMBER']:
    if '-' in each:
        temp=re.findall('(\d*)-(\d*)',str(each))
        for i in range(int(temp[0][0]),int(temp[0][1])+1):
            temp_df = pd.DataFrame(rochester.loc[rochester['NUMBER'] == str(each)])
            temp_df = pd.DataFrame(temp_df)
            temp_df['NUMBER'] = int(i)
            rochester = pd.concat([rochester,temp_df], axis = 0)
            rochester.reset_index(inplace = True, drop = True)
        rochester = rochester.drop(rochester[rochester['NUMBER'] == each].index)

ValueError: invalid literal for int() with base 10: ''

### Zillow Estimator
At this point we have a full list of addresses and postal codes in our dataframe which we can use to enter into the Zillow API.  This forloop allows for some flexibility in the requests, if Zillow has no estimate on a property the loop should add a "None" value to the list.

In [1]:
from progressbar import ProgressBar
bar = ProgressBar()

In [9]:
zestimate_list = []
for i in bar(range(len(rochester[:500]))):
    try:
        address = str(rochester['NUMBER'][i]+' '+rochester['STREET'][i]+'.,'+rochester['CITY'][i]+', NY')
        postal_code = str(int(rochester['POSTCODE'][i]))
        data = api.GetSearchResults(zid,address,postal_code)
        zestimate_list.append(data.zestiamte.amount)
    except:
        zestimate_list.append('None')
    
print('Missing Values:',len(rochester[:500])-len(zestimate_list))
    

100% (500 of 500) |######################| Elapsed Time: 0:01:57 Time:  0:01:57


Missing Values: 0


In [10]:
zestimate_list

[86896,
 86956,
 82596,
 93138,
 None,
 60778,
 'None',
 86826,
 75063,
 55659,
 52596,
 51933,
 56074,
 46318,
 64887,
 'None',
 'None',
 'None',
 74107,
 94059,
 82271,
 85861,
 65770,
 61710,
 None,
 65734,
 'None',
 66232,
 80605,
 57538,
 44647,
 47480,
 57390,
 56351,
 40521,
 48114,
 67341,
 39145,
 37874,
 87579,
 None,
 'None',
 144768,
 238587,
 177408,
 'None',
 78167,
 86268,
 'None',
 103214,
 77672,
 'None',
 77373,
 None,
 None,
 78713,
 208590,
 'None',
 117571,
 None,
 123104,
 'None',
 'None',
 79058,
 113427,
 90958,
 79110,
 119500,
 None,
 68022,
 59607,
 50596,
 73847,
 84100,
 61411,
 57702,
 58177,
 61273,
 57375,
 63245,
 69033,
 51387,
 56175,
 60665,
 113697,
 None,
 59435,
 117149,
 117576,
 93943,
 112168,
 87707,
 51213,
 72505,
 72756,
 74489,
 72780,
 71831,
 69936,
 117734,
 68229,
 115561,
 76937,
 75476,
 235894,
 149480,
 91047,
 88198,
 173634,
 68105,
 57065,
 121104,
 43695,
 74921,
 95960,
 56489,
 45410,
 109439,
 88035,
 45603,
 84223,
 60030,


In [ ]:
rochester['Zestimate'] = 